# Démonstration de balance des blancs (white balancing)
1. manuelle
2. automatique avec grey world
3. automatique avec white world

## Correction de couleur manuelle -- sélection d'une région grise

In [ ]:
# settings for display, wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# import the libs
import matplotlib as mpl
mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)
import matplotlib.pyplot as plt
import numpy as np

from roi import new_ROI

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from imageio import imread
img = imread('wb-reelle.jpg')/255.0
from skimage.transform import resize
img = resize(img, [600, 900], mode='reflect', anti_aliasing=True)

In [ ]:
# demandons à l'utilisateur d'identifier une région correspondant à un objet neutre
%matplotlib notebook
fig = plt.figure(figsize=(10,5), num='Draw ROI')
im_axis = plt.imshow(img);
plt.tight_layout()
roi = new_ROI(im_axis, shape='polygon')

In [ ]:
coordinates = roi.get_indices()
assert coordinates is not None, 'roi is empty'

In [ ]:
# coordiantes to indices
indices = np.ravel_multi_index(coordinates, dims=img.shape[0:2])

In [ ]:
# truc de pro: vectoriser l'image en un vecteur de Nx3 (N = # de pixels)
imgVec = img.reshape([-1, 3])
offset = np.mean(imgVec[indices, :], axis=None) - np.mean(imgVec[indices, :], axis=0)

In [ ]:
# appliquons cet «offset» à l'image. 
imgVecWbManu = imgVec + offset.reshape((1,3))

In [ ]:
# validons que la moyenne de l'image _sous le masque_ est effectivement neutre
newMean = np.mean(imgVecWbManu[indices, :], 0)
assert np.all(np.abs(newMean - np.mean(newMean)) < 1e-10), 'error de moyenne'
print('Moyenne sous le masque: ')
print(newMean)

In [ ]:
# re-convertissons en format "image"
imgWbManu = img + offset.reshape((1,1,3))
imgWbManu = np.maximum(0, np.minimum(1, imgWbManu))
fig = plt.figure(figsize=(10,5), num='White balanced')
plt.imshow(imgWbManu);

## Correction de couleur automatique -- "grey world"

In [ ]:
img = imread('wb-reelle.jpg').astype('float')/255.0
img = resize(img, [600, 900], mode='reflect')

In [ ]:
# nous faisons l'hypothèse que la moyenne des pixels devrait être grise (R=G=B)
# calculons tout d'abord la moyenne de l'image
# calculer la moyenne est alors super simple!
moyenne = np.mean(img, (0,1))
# nous voulons que R=G=B. donc trouvons un «offset» qui mettra
# tous les canaux au meme niveau, disons G
offset = np.mean(moyenne) - moyenne

In [ ]:
# appliquons cet «offset» à l'image. re-convertissons en format "image"
imgWbGray = img + offset.reshape((1,1,3))
# imgWbGray = np.maximum(0, np.minimum(1, imgWbGray))

In [ ]:
# validons que la moyenne de l'image est effectivement neutre
newMean = np.mean(imgWbGray, (0,1))
assert np.all(np.abs(newMean - np.mean(newMean)) < 1e-10), 'error de moyenne'
print('Moyenne de l''image: ')
print(newMean)

## Correction de couleur automatique -- "white world"

In [ ]:
img = imread('wb-reelle.jpg').astype('float')/255
img = resize(img, [600, 900], mode='reflect', anti_aliasing=True)

In [ ]:
# nous faisons l'hypothèse que le pixel le plus brilliant devrait être
# blanc (R=G=B encore, mais valeur plus élevée donc proche de blanc)
N = 10
# trouvons les N pixels les plus brilliants (sans toutefois être saturé!)
# faisons l'hypothèse que tout ce qui est au dessus de ce seuil est saturé
seuilSat = 250/255; 
diff = seuilSat - img;
diff[diff<=0] = 1;
# chaque canal stocke la distance entre le pixel et "blanc". plus la
# distance est basse, plus le pixel est proche de blanc. 
diff = np.max(diff, -1);
print(diff.shape)

In [ ]:
sind = np.argsort(diff.flatten())
sind = sind[0:N]

In [ ]:
# cette couleur est la couleur la plus brillante, et pas saturée
couleur = np.mean(img.reshape(-1,3)[sind, :], 0);
print(couleur)

# nous voulons que R=G=B. donc trouvons un facteur d'echelle qui mettra
# tous les canaux au meme niveau, disons G
# facteur = squeeze(couleur(2) ./ couleur)';
offset = couleur[1] - couleur

In [ ]:
# appliquons ce facteur à l'image. 
imgWbWhite = img + offset.reshape((1,1,3))
imgWbWhite = np.maximum(0, np.minimum(1, imgWbWhite))

In [ ]:
# validons que la moyenne de l'image est effectivement neutre
newMean = np.mean(imgWbWhite.reshape(-1,3)[sind, :], axis=0)
assert np.all(np.abs(newMean - np.mean(newMean)) < 1e-10), 'error de moyenne'
print('Moyenne de l''image: ')
print(newMean)

## Affichons les résultats des 3 algorithmes

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,5), sharex=True, sharey=True)
ax[0][0].imshow(img); ax[0][0].set_title('Image originale')
ax[0][1].imshow(imgWbManu); ax[0][1].set_title('Image équilibrée manuellement');
ax[1][0].imshow(imgWbGray); ax[1][0].set_title('Grey world');
ax[1][1].imshow(imgWbWhite); ax[1][1].set_title('White world');